In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
import os
print(os.environ.get('KITE_API_KEY'))
print(os.environ.get('KITE_SECRET_KEY'))

9kxpk63gv7k76agn
zkgtjepzhmwrmpd9rmhgecoeuntxfk54


In [3]:
import pandas as pd
import tqdm
import matplotlib.pyplot as plt
import math

In [4]:
from maestro.kite.connector.kite_connector import KiteConnector
from maestro.kite.historical.fetch_historical_data import HistoricalDataFetcher, Interval, get_today, get_date_before_today
from maestro.kite.instruments.kite_instruments import TradingInstruments, ExchangeType

In [10]:
connector = KiteConnector()

Goto this url and get the request token from the redirect link: https://kite.zerodha.com/connect/login?api_key=9kxpk63gv7k76agn&v=3


In [11]:
instruments = TradingInstruments(connector, live=True)

In [12]:
nse_instruments = instruments.get_instruments_by_exchange(ExchangeType.NSE)
print(len(nse_instruments))

5414


In [13]:
history_fetcher = HistoricalDataFetcher(connector)

In [14]:
out_dict = {}

In [26]:
import datetime
from_date = (datetime.date.today() - datetime.timedelta(days=30)).strftime("%Y-%m-%d")
to_date = datetime.date.today().strftime("%Y-%m-%d")
#for instrument in tqdm.tqdm(nse_instruments):
for val in [True]:
    #print(instrument)
    #token = instrument["instrument_token"]
    token = 738561 # for reliance
    #if token in out_dict:
    #    continue
    prices = history_fetcher.fetch_historical_data(token, from_date, to_date, Interval.MINUTE)
    #print(token, prices)
    #break
#     out_dict[token] = prices

In [27]:
data = pd.DataFrame(prices)

In [35]:
import backtrader as bt
row = data.iloc[0]
bt.date2num(row['date'].to_pydatetime())
#bt.date2num(row.name.to_pydatetime())

738638.15625

In [ ]:
total_days = 360
days_per_file = 30
to_date = get_today()

for i in range(30, total_days, days_per_file):
    
    print(i)
    from_date = get_date_before_today(i)
    name = ('-').join(from_date.split('-')[:-1]).replace('-', '_')[2:]
    print(from_date, to_date, name)
    to_date = from_date


In [ ]:
import pickle
with open("/Users/akashanand/repo/maestro/data/data.pickle", "wb") as f:
    pickle.dump(out_dict, f)

In [ ]:
# Instrument wise data with money flow
instrument_wise_cash_flow = []
for instrument in tqdm.tqdm(out_dict.keys()):
    temp = []
    for row in out_dict[instrument]:
        temp.append(row["close"] * row["volume"])
    if len(temp) > 0:
        flow = sum(temp)/len(temp)
    else:
        flow = 0
    instrument_wise_cash_flow.append({"instrument": instrument, "flow": flow})



In [ ]:
# Sort by cash flow
instrument_wise_cash_sorted = sorted(instrument_wise_cash_flow, key=lambda x: x["flow"], reverse=True)
print("All instruments: ", len(instrument_wise_cash_sorted))
instrument_wise_cash_sorted = [x for x in instrument_wise_cash_sorted if x["flow"] > 0]
print("Positive instruments: ", len(instrument_wise_cash_sorted))
# Sample 500 random instruments
import random
random.seed(42)
instrument_wise_cash_sorted = sorted(random.sample(instrument_wise_cash_sorted, 500), key=lambda x: x["flow"], reverse=True)


In [ ]:
small_dict = {}
for instrument in instrument_wise_cash_sorted:
    small_dict[instrument["instrument"]] = out_dict[instrument["instrument"]]

import pickle
with open("/Users/akashanand/repo/maestro/data/data_small.pickle", "wb") as f:
    pickle.dump(small_dict, f)

In [ ]:
# Plot the cash flow in log scale
plt.plot([math.log(x["flow"]) for x in instrument_wise_cash_sorted if x["flow"] > 0])

In [ ]:
import pandas as pd
pd_rel = pd.DataFrame(rel)
pd_rel

In [ ]:
pd_nse["exchange"].value_counts()

In [ ]:
pd_nse["segment"].value_counts()

In [ ]:
pd_nse["instrument_type"].value_counts()

In [ ]:
pd_nse.head()